In [1]:
import os
from typing import List, Tuple
import numpy as np
from PIL import Image
import cv2


def extract_frames_from_video(video_path: str, output_dir: str, frame_interval: int = 60) -> None:
    """
    Extracts frames from a video file at a specified interval and saves them to a directory.

    Args:
        video_path (str): Path to the input video file.
        output_dir (str): Path to the output directory for saving the extracted frames.
        frame_interval (int): Interval (in frames) at which frames are extracted. Default is 60 frames.

    Returns:
        None
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Initialize the frame counter
    frame_count = 0

    # Loop through the frames and save every frame_interval-th frame
    while True:
        ret, frame = video.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_path = os.path.join(output_dir, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)

        frame_count += 1

    # Release the video capture object
    video.release()

def convert_to_grayscale(images: List[np.ndarray]) -> List[np.ndarray]:
    """
    Converts a list of RGB images to grayscale.

    Args:
        images (List[np.ndarray]): List of RGB images.

    Returns:
        List[np.ndarray]: List of grayscale images.
    """
    gray_images = []
    for image in images:
        gray_image = np.dot(image[..., :3], [0.299, 0.587, 0.114])
        gray_images.append(gray_image)
    return gray_images

def enhance_images(gray_images: List[np.ndarray]) -> List[np.ndarray]:
    """
    Enhances a list of grayscale images using contrast stretching.

    Args:
        gray_images (List[np.ndarray]): List of grayscale images.

    Returns:
        List[np.ndarray]: List of enhanced grayscale images.
    """
    enhanced_images = []
    for image in gray_images:
        min_val = np.min(image)
        max_val = np.max(image)
        enhanced_image = (image - min_val) / (max_val - min_val) * 255
        enhanced_image = np.clip(enhanced_image, 0, 255).astype(np.uint8)
        enhanced_images.append(enhanced_image)
    return enhanced_images

def detect_features(images: List[np.ndarray]) -> List[List[Tuple[int, int]]]:
    """
    Detects features in a list of grayscale images using the Harris corner detector.

    Args:
        images (List[np.ndarray]): List of grayscale images.

    Returns:
        List[List[Tuple[int, int]]]: List of lists of keypoints, where each inner list corresponds to the keypoints detected in the corresponding image.
    """
    feature_list = []
    for image in images:
        keypoints = harris_corner_detector(image)
        feature_list.append(keypoints)
    return feature_list

def harris_corner_detector(image: np.ndarray, k: float = 0.04) -> List[Tuple[int, int]]:
    """
    Implements the Harris corner detector algorithm.

    Args:
        image (np.ndarray): Grayscale image.
        k (float): Sensitivity factor. Default is 0.04.

    Returns:
        List[Tuple[int, int]]: List of keypoints (x, y) coordinates.
    """
    # Compute image derivatives
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

    # Compute Harris response matrix
    Ixx = sobelx ** 2
    Ixy = sobelx * sobely
    Iyy = sobely ** 2

    # Compute Harris corner response
    harris_response = (Ixx * Iyy - Ixy ** 2) - k * ((Ixx + Iyy) ** 2)

    # Find keypoints
    keypoints = []
    for y in range(harris_response.shape[0]):
        for x in range(harris_response.shape[1]):
            if harris_response[y, x] > 0.01 * harris_response.max():
                keypoints.append((x, y))

    return keypoints

def match_features_across_pairs(feature_list: List[List[Tuple[int, int]]]) -> List[List[Tuple[Tuple[int, int], Tuple[int, int]]]]:
    """
    Matches features across pairs of images using a simple brute-force matching approach.

    Args:
        feature_list (List[List[Tuple[int, int]]]): List of lists of keypoints, where each inner list corresponds to the keypoints detected in the corresponding image.

    Returns:
        List[List[Tuple[Tuple[int, int], Tuple[int, int]]]]: List of lists of matched feature pairs, where each inner list corresponds to the matches between the corresponding pair of images.
    """
    matched_features = []
    for i in range(len(feature_list) - 1):
        kp1, kp2 = feature_list[i], feature_list[i + 1]
        matches = brute_force_match(kp1, kp2)
        matched_features.append(matches)
    return matched_features

def brute_force_match(keypoints1: List[Tuple[int, int]], keypoints2: List[Tuple[int, int]]) -> List[Tuple[Tuple[int, int], Tuple[int, int]]]:
    """
    Performs brute-force matching between two sets of keypoints.

    Args:
        keypoints1 (List[Tuple[int, int]]): List of keypoints for the first image.
        keypoints2 (List[Tuple[int, int]]): List of keypoints for the second image.

    Returns:
        List[Tuple[Tuple[int, int], Tuple[int, int]]]: List of matched feature pairs.
    """
    matches = []
    for kp1 in keypoints1:
        min_dist = float('inf')
        best_match = None
        for kp2 in keypoints2:
            dist = np.linalg.norm(np.array(kp1) - np.array(kp2))
            if dist < min_dist:
                min_dist = dist
                best_match = kp2
        if best_match is not None:
            matches.append((kp1, best_match))
    return matches

ModuleNotFoundError: No module named 'cv2'